In [ ]:
from utils.portfolio import Portfolio
from utils.tickers import tickers, allocation
from tqdm import tqdm
import numpy as np
import plotly.graph_objects as go
import pandas as pd

start = "2023-01-01"  # TBD
end = "2024-12-31"

portfolio = Portfolio(tickers, start, end)

## Initial portfolio and allocations

In [2]:
current_assets = portfolio.get_assets()
current_allocations = allocation

In [ ]:
portfolio.get_asset("NVDA").get_data().tail(3)

,Open,High,Low,Close,Volume,EMA_50,EMA_200,RSI,ATR,Lower_Band,BBM_20_2.0,Upper_Band,BBB_20_2.0,BBP_20_2.0,%K,%D,MACD,MACDh_12_26_9,Signal,OBV
Date,,,,,,,,,,,,,,,,,,,,
2024-12-26 00:00:00-05:00,139.687155,140.837058,137.717335,139.917130,116205600,136.462118,115.739915,56.271493,4.954448,128.343392,137.304966,146.266540,13.053532,0.645742,67.450453,55.500637,-0.773720,0.431834,-1.205554,2.969486e+10
2024-12-27 00:00:00-05:00,138.537258,139.007216,134.697615,136.997391,170582600,136.483109,115.951432,49.502497,4.992315,128.472025,137.388924,146.305824,12.980522,0.478045,68.248858,64.434774,-0.699350,0.404963,-1.104313,2.952428e+10
2024-12-30 00:00:00-05:00,134.817597,140.257099,134.007674,137.477356,167734700,136.522099,116.165621,50.588148,5.171902,128.442627,137.351404,146.260180,12.972239,0.507069,69.179529,68.292947,-0.594824,0.407591,-1.002415,2.969202e+10


## Trading Strategies 

In [4]:
# Trading Strategy 1: Simple scalping based on this video https://www.youtube.com/watch?v=C3bh6Y4LpGs

# Trend detection
## Uptrend (EMA50>EMA200) - long positions
## Downtrend - short positions
# Bollinger band edges for entry signals
## During a uptrend, if price crosses lower bollinger curve, open a long position 
## During a downtrend, if price crosses upper bollinger band, open a short position 
## Stop-Loss (SL) = slcoef * ATR
## Take Profit (TP) = TPSL * SL 

In [ ]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider - test for multiple backcandles (e.g. 1 week) to prevent fitting to noise
    start = max(0, current_candle - backcandles) 
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_50"] < relevant_rows["EMA_200"]):
        return 1
    elif all(relevant_rows["EMA_50"] > relevant_rows["EMA_200"]):
        return 2
    else:
        return 0


df= portfolio.get_asset("NVDA").get_data() # Run only on NVDA for now 
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1)

100%|██████████| 501/501 [00:00<00:00, 2819.49it/s]


In [8]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['Lower_Band'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2 #Long signal 
    
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['Upper_Band'][current_candle]
        #and df.RSI[current_candle]>40
        ):
            
            return 1 #Short signal 
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 501/501 [00:00<00:00, 1477.43it/s]


In [24]:
df[df.TotalSignal != 0].head(20)

,Date,Open,High,Low,Close,Volume,EMA_50,EMA_200,RSI,ATR,...,BBP_20_2.0,%K,%D,MACD,MACDh_12_26_9,Signal,OBV,EMASignal,TotalSignal,pointpos
325,2024-04-19 00:00:00-04:00,83.122695,84.296305,75.581173,76.174973,875198000,81.896498,59.779844,28.121929,4.395506,...,-0.244545,12.677095,21.271107,-0.586502,-1.505464,0.918962,2.638635e+10,2,2,75.580173
390,2024-07-24 00:00:00-04:00,119.140633,119.920440,113.412050,114.221848,327776900,116.790522,85.414942,37.361860,5.807272,...,-0.064166,23.326046,23.236758,-0.006848,-1.886687,1.879839,2.908666e+10,2,2,113.411050
391,2024-07-25 00:00:00-04:00,113.012149,116.601261,106.273812,112.252335,460067000,116.612554,85.681980,35.369369,6.453011,...,-0.047138,18.127067,21.973795,-0.816864,-2.157362,1.340498,2.862659e+10,2,2,106.272812
394,2024-07-30 00:00:00-04:00,111.492519,111.962404,102.514736,103.704445,486833300,115.786467,86.383016,27.641246,6.507794,...,-0.090274,14.636369,16.728650,-2.952801,-2.564312,-0.388489,2.818501e+10,2,2,102.513736


In [9]:
# Create points above and below candles when signal is detected 
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [ ]:
# Plot candlestick

## Backtesting and evaluation

Use some metrics (e.g Sharpe ratio) to compare strategies - to be added